In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from qutip import *
from qutip.piqs import *
from matplotlib import cm
from qutip.fastsparse import fast_csr_matrix, fast_identity
import scipy.sparse as sparse
import scipy.linalg as linalg


In [81]:
# First we define single spin-1 operators, dimension is 3^N
N = 2
I2 = qeye(3)
f_z = []
f_p = []
f_m = []

for jj in range(N):
    f_z.append(tensor([ I2 ] * jj + [ jmat(1,"z") ] + [ I2 ] * (N-jj-1)))
    f_p.append(tensor([ I2 ] * jj + [ jmat(1, "+") ] + [ I2 ] * (N-jj-1)))
    f_m.append(tensor([ I2 ] * jj + [ jmat(1, "-") ] + [ I2 ] * (N-jj-1)))
    
#Collective spin-1 operators, the dimension goes as 3^N
Fz = sum(f_z)
Fp = sum(f_p)
Fm = sum(f_m)
Fz

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[ 2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -2.]]

In [19]:
# Construct the Spin-1 algebra

def spin_1(N, op=None):
    
    """Create the list [fx, fy, fz] with the spin operators.
    The operators are constructed for a collection of N three-level systems
    (TLSs). Each element of the list, i.e., fx, is a vector of `qutip.Qobj`
    objects (spin matrices), as it cointains the list of the SU(3) Gell Mann
    matrices for the N TLSs. Each TLS operator fx[i], with i = 0, ..., (N-1),
    is placed in a :math:`3^N`-dimensional Hilbert space.
    Notes
    -----
    fx[i] is :math:`\\frac{\\sigma_x}{2}` in the composite Hilbert space.
    Parameters
    ----------
    N: int
        The number of spin 1 atoms.
    Returns
    -------
    spin_operators: list or :class: qutip.Qobj
        A list of `qutip.Qobj` operators - [fx, fy, fz] or the
        requested operator.
    """
    #Define N spin-1 atoms in uncoupled basis
    N = int(N)
    fx = [0 for i in range(N)]
    fy = [0 for i in range(N)]
    fz = [0 for i in range(N)]
    fp = [0 for i in range(N)]
    fm = [0 for i in range(N)]
    
    fx[0] = jmat(1, "x")
    fy[0] = jmat(1, "y")
    fz[0] = jmat(1, "z")
    fp[0] = jmat(1, "+")
    fm[0] = jmat(1, "-")
    
    #Place operators on the N-atom Hilbert-Space
    
    for j in range(N-1):
        fx[0] = tensor(fx[0], identity(3))
        fy[0] = tensor(fy[0], identity(3))
        fz[0] = tensor(fz[0], identity(3))
        fp[0] = tensor(fp[0], identity(3))
        fm[0] = tensor(fm[0], identity(3))
    
    #Create the operators
    a = [i for i in range(N)]
    b = [[a[i - i2] for i in range(N)] for i2 in range(N)]
    
    for i in range(1,N):
        fx[i] = fx[0].permute(b[i])
        fy[i] = fy[0].permute(b[i])
        fz[i] = fz[0].permute(b[i])
        fp[i] = fp[0].permute(b[i])
        fm[i] = fm[0].permute(b[i])
    
    spin_1 = [fx, fy, fz]
    
    if not op:
        return spin_1
    elif op == 'x':
        return fx
    elif op == 'y':
        return fy
    elif op == 'z':
        return fz
    elif op == '+':
        return fp
    elif op == '-':
        return fm
    else:
        raise TypeError('Invalid type')
    

In [116]:
#Construct the Collective-Spin operators

def spin1(N, op=None):
    """Construct the the collective spin algebra in the uncoupled basis.
    Fx, Fy, Fz, Fp, Fm are constructed in the uncoupled basis of the
    spin-1 system (TLS). Each collective operator is placed in a
    Hilbert space of dimension 3^N.
    Parameters
    ----------
    N: int
        The number of spin-1 systems.
    op: str
        The operator to return 'x','y','z','+','-'.
        If no operator given, then output is the list of operators
        for ['x','y','z',].
    Returns
    -------
    collective_operators: list or :class: qutip.Qobj
        A list of `qutip.Qobj` representing all the operators in
        uncoupled" basis or a single operator requested.
    """
    
    N = int(N)
    
    fx, fy, fz = spin_1(N)
    fp, fm = spin_1(N, "+"), spin_1(N, "-")
    
    
    Fx = sum(fx)
    Fy = sum(fy)
    Fz = sum(fz)
    Fp = sum(fp)
    Fm = sum(fm)
    
    collective_operators = [Fx, Fy, Fz, Fp, Fm]
    
    if not op:
        return collective_operators
    elif op == 'x':
        return Fx
    elif op == 'y':
        return Fy
    elif op == 'z':
        return Fz
    elif op == '+':
        return Fp
    elif op == '-':
        return Fm
    else:
        raise TypeError('Invalid type')
        
    return Fx, Fy, Fz, Fp, Fm

In [122]:
spin1(2,"x")

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[0.         0.70710678 0.         0.70710678 0.         0.
  0.         0.         0.        ]
 [0.70710678 0.         0.70710678 0.         0.70710678 0.
  0.         0.         0.        ]
 [0.         0.70710678 0.         0.         0.         0.70710678
  0.         0.         0.        ]
 [0.70710678 0.         0.         0.         0.70710678 0.
  0.70710678 0.         0.        ]
 [0.         0.70710678 0.         0.70710678 0.         0.70710678
  0.         0.70710678 0.        ]
 [0.         0.         0.70710678 0.         0.70710678 0.
  0.         0.         0.70710678]
 [0.         0.         0.         0.70710678 0.         0.
  0.         0.70710678 0.        ]
 [0.         0.         0.         0.         0.70710678 0.
  0.70710678 0.         0.70710678]
 [0.         0.         0.         0.         0.         0.70710678
  0.         0.70710678 0.        ]]

In [93]:
basis(3,2)

Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]]

In [113]:
jmat(1)

(Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
 Qobj data =
 [[0.         0.70710678 0.        ]
  [0.70710678 0.         0.70710678]
  [0.         0.70710678 0.        ]],
 Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
 Qobj data =
 [[0.+0.j         0.-0.70710678j 0.+0.j        ]
  [0.+0.70710678j 0.+0.j         0.-0.70710678j]
  [0.+0.j         0.+0.70710678j 0.+0.j        ]],
 Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
 Qobj data =
 [[ 1.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0. -1.]])